You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan Document:

Title: Unveiling the Latest Trends in Artificial Intelligence

Introduction:
Artificial Intelligence (AI) is revolutionizing various industries with its capabilities to enhance efficiency, productivity, and decision-making processes. In this blog article, we will explore the latest trends, key players, and noteworthy news in the field of Artificial Intelligence.

Key Points:
1. Latest Trends in AI:
- Machine Learning algorithms are becoming more advanced and efficient, leading to improv

I now can give a great answer

Final Answer:
# Unveiling the Latest Trends in Artificial Intelligence

## Introduction
Artificial Intelligence (AI) has been at the forefront of technological advancements, reshaping industries and revolutionizing the way we work and live. In this blog post, we will delve into the latest trends, key players, and noteworthy news in the dynamic world of AI.

## Latest Trends in AI
One of the most prominent trends in AI is the evolution of Machine Learning algorithms, which are becoming more sophisticated and efficient. This progress has led to significant improvements in predictive analytics and personalized recommendations, enhancing user experiences across various platforms. Additionally, Natural Language Processing (NLP) has gained traction in applications such as chatbots, virtual assistants, and sentiment analysis, enabling more seamless interactions between humans and machines. AI-powered automation is another key trend, streamlining processes in ind

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Latest Trends in Artificial Intelligence

## Introduction
Artificial Intelligence (AI) has been at the forefront of technological advancements, reshaping industries and revolutionizing the way we work and live. In this blog post, we will delve into the latest trends, key players, and noteworthy news in the dynamic world of AI.

## Latest Trends in AI
One of the most prominent trends in AI is the evolution of Machine Learning algorithms, which are becoming more sophisticated and efficient. This progress has led to significant improvements in predictive analytics and personalized recommendations, enhancing user experiences across various platforms. Additionally, Natural Language Processing (NLP) has gained traction in applications such as chatbots, virtual assistants, and sentiment analysis, enabling more seamless interactions between humans and machines. AI-powered automation is another key trend, streamlining processes in industries like healthcare, finance, and manufacturing, ultimately increasing productivity and efficiency. Edge AI, on the other hand, is empowering devices to perform complex computations locally, reducing latency and ensuring data privacy.

## Key Players in AI
In the realm of AI, several key players have emerged as pioneers in research and development. Google's DeepMind stands out for its breakthroughs in reinforcement learning and neural networks, pushing the boundaries of AI capabilities. IBM Watson offers cognitive computing solutions for businesses across various sectors, while Amazon Web Services (AWS) provides a range of AI services, from image recognition to language translation. Microsoft Azure, on the other hand, equips developers and businesses with AI tools like Azure Machine Learning and Cognitive Services, driving innovation in the industry.

## Noteworthy News in AI
Recent developments in AI have captured the attention of tech enthusiasts and industry professionals alike. OpenAI's GPT-3 model has garnered praise for its ability to generate human-like text and enhance natural language understanding, opening up new possibilities for AI applications. NVIDIA's advancements in AI hardware, including GPUs and accelerators, have fueled innovation in deep learning applications, pushing the boundaries of what AI can achieve. Tesla's Autopilot technology continues to lead the way in autonomous driving, leveraging AI-powered features to enhance vehicle safety and efficiency.

## Conclusion
As we navigate the ever-evolving landscape of Artificial Intelligence, it is crucial to stay informed about the latest trends, key players, and groundbreaking news in the field. By leveraging the power of AI, businesses and individuals can unlock new opportunities for growth and innovation, driving progress and shaping the future of technology.

By following this comprehensive content plan, this blog post aims to provide valuable insights and information to our target audience of technology enthusiasts, business professionals, students, and individuals interested in the latest developments in Artificial Intelligence. Stay tuned for more updates on the exciting world of AI.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on YOUR TOPIC HERE.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: Exploring the Latest Trends in Sustainable Fashion

Introduction:
- Brief overview of the growing importance of sustainable fashion in the fashion industry
- Mention of the key players leading the charge towards sustainability in fashion

Key Points:
1. Latest Trends in Sustainable Fashion:
   - Use of eco-friendly materials such as organic cotton, hemp, and recycled polyester
   - Adoption of circular fashion practices like clothing rental and resale
   - Emphasis on transparency and traceability in the supp

I now can give a great answer.

Final Answer: 

# Exploring the Latest Trends in Sustainable Fashion

In recent years, the fashion industry has witnessed a significant shift towards sustainability, with more brands and consumers embracing eco-friendly practices and ethical production methods. This movement towards sustainable fashion is driven by the growing awareness of the environmental and social impact of the industry. Key players in the fashion world are leading the charge towards sustainability, setting new trends and inspiring others to follow suit.

## Latest Trends in Sustainable Fashion

One of the prominent trends in sustainable fashion is the use of eco-friendly materials such as organic cotton, hemp, and recycled polyester. These materials not only reduce the environmental footprint of clothing production but also promote a more ethical supply chain. Another trend gaining traction is the adoption of circular fashion practices, including clothing rental and resale. By exten

In [14]:
Markdown(result)

# Exploring the Latest Trends in Sustainable Fashion

In recent years, the fashion industry has witnessed a significant shift towards sustainability, with more brands and consumers embracing eco-friendly practices and ethical production methods. This movement towards sustainable fashion is driven by the growing awareness of the environmental and social impact of the industry. Key players in the fashion world are leading the charge towards sustainability, setting new trends and inspiring others to follow suit.

## Latest Trends in Sustainable Fashion

One of the prominent trends in sustainable fashion is the use of eco-friendly materials such as organic cotton, hemp, and recycled polyester. These materials not only reduce the environmental footprint of clothing production but also promote a more ethical supply chain. Another trend gaining traction is the adoption of circular fashion practices, including clothing rental and resale. By extending the lifecycle of garments and reducing waste, circular fashion contributes to a more sustainable industry. Moreover, there is an increasing emphasis on transparency and traceability in the supply chain, allowing consumers to make more informed purchasing decisions.

## Key Players in Sustainable Fashion

Leading the way in sustainable fashion are key players such as Patagonia, known for its commitment to sustainability and environmental activism. Patagonia's innovative approach to eco-friendly materials and ethical production has set a benchmark for the industry. Stella McCartney is another prominent figure in luxury sustainable fashion, combining style and sustainability seamlessly. Eileen Fisher, a pioneer in ethical and sustainable fashion practices, advocates for mindful consumption and responsible production methods.

## Noteworthy News in Sustainable Fashion

Recent collaborations between sustainable fashion brands and mainstream retailers have brought sustainable fashion into the spotlight, making it more accessible to a wider audience. Success stories of sustainable fashion startups making a mark in the industry showcase the growing demand for ethical and eco-conscious clothing. Legislative changes and initiatives promoting sustainability in fashion are also shaping the industry's future, encouraging brands to prioritize environmental and social responsibility.

By exploring the latest trends, key players, and noteworthy news in sustainable fashion, we can gain a deeper understanding of the importance of making conscious choices in our fashion consumption. As millennials and Gen Z consumers, fashion enthusiasts, and industry professionals, we have the power to drive positive change in the fashion industry. Let's continue to support sustainable fashion brands and initiatives, shaping a more sustainable and ethical future for fashion. 

Remember, the choices we make today can have a lasting impact on the planet and future generations. Let's embrace sustainable fashion and be part of the solution.

For further reading and exploration of sustainable fashion practices, check out the Sustainable Apparel Coalition, Fashion Revolution, and The True Cost documentary. Join the movement towards a more sustainable fashion industry today!

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).